In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [ ]:
#loading data
wiki = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/wiki_raw.csv')
kaggle = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/kaggle_raw.csv')

In [ ]:
wiki

,id,url,title,wiki_intro,generated_intro,title_len,wiki_intro_len,generated_intro_len,prompt,generated_text,prompt_tokens,generated_text_tokens
0,63064638,https://en.wikipedia.org/wiki/Sexhow%20railway...,Sexhow railway station,Sexhow railway station was a railway station b...,Sexhow railway station was a railway station l...,3,174,78,200 word wikipedia style introduction on 'Sexh...,"located in the town of Sexhow, on the Cumbria...",25,88
1,279621,https://en.wikipedia.org/wiki/Eti%C3%A4inen,Etiäinen,"In Finnish folklore, all places and things, an...","In Finnish folklore, all places and things, an...",1,187,80,200 word wikipedia style introduction on 'Etiä...,"animate or inanimate, have a spirit or ""etiäi...",26,101
2,287229,https://en.wikipedia.org/wiki/Inverse%20functi...,Inverse function theorem,"In mathematics, specifically differential calc...","In mathematics, specifically differential calc...",3,170,59,200 word wikipedia style introduction on 'Inve...,function theorem states that for every real-v...,26,65
3,26712375,https://en.wikipedia.org/wiki/Stepping%20on%20...,Stepping on Roses,is a Japanese shōjo manga series written and i...,is a Japanese shōjo manga series written and i...,3,335,121,200 word wikipedia style introduction on 'Step...,and illustrated by Maki Fujii. The series fol...,26,150
4,38894426,https://en.wikipedia.org/wiki/Rob%20Bradley,Rob Bradley,"Robert Milner ""Rob"" Bradley, Jr. (born August ...","Robert Milner ""Rob"" Bradley, Jr. (born August ...",2,170,136,200 word wikipedia style introduction on 'Rob ...,"29, 1973) is an American former professional ...",28,162
...,...,...,...,...,...,...,...,...,...,...,...,...
149995,44173767,https://en.wikipedia.org/wiki/Randy%20Borum,Randy Borum,Randy Borum is a Professor and Coordinator of ...,Randy Borum is a Professor and Coordinator of ...,2,185,71,200 word wikipedia style introduction on 'Rand...,of the Master of Fine Arts Program in Creativ...,25,92
149996,33564134,https://en.wikipedia.org/wiki/Sa%27och%20language,Sa'och language,"Sa'och (, also, ""Sauch"") is an endangered, nea...","Sa'och (, also, ""Sauch"") is an endangered, nuc...",2,175,134,200 word wikipedia style introduction on 'Sa'o...,"nuclear-speaking, isolate language of the Ath...",33,184
149997,4219548,https://en.wikipedia.org/wiki/Philip%20Hanawalt,Philip Hanawalt,Philip C. Hanawalt (born 1931) is an American ...,Philip C. Hanawalt (born 1931) is an American ...,2,166,191,200 word wikipedia style introduction on 'Phil...,American graphic artist and illustrator. He i...,30,272
149998,2625970,https://en.wikipedia.org/wiki/Vossius%20Gymnasium,Vossius Gymnasium,Vossius Gymnasium is a public gymnasium in Ams...,Vossius Gymnasium is a public gymnasium in the...,2,168,108,200 word wikipedia style introduction on 'Voss...,"the town of Vossius, Netherlands. It is named...",32,147


In [ ]:
#random sampling to reduce the dataset into 10K rows to reduce the computational power required for model processing
np.random.seed(42)
# Sample about 10K rows from the wikipedia dataset randomly
wiki = wiki.sample(frac= 0.25*len(kaggle)/len(wiki))
# Create a new DataFrame to store concatenated data
wiki_data = pd.DataFrame(columns=['text', 'label'])
# Fill 'text' column with 'wiki_intro' and 'generated_intro' content concatenated
wiki_data['text'] = pd.concat([wiki['wiki_intro'], wiki['generated_intro']], ignore_index=True)
# Fill 'label' column with 0 for 'wiki_intro' and 1 for 'generated_intro'
wiki_data['label'] = np.concatenate((np.zeros(len(wiki['wiki_intro'])), np.ones(len(wiki['generated_intro']))))
# check if there're missing values in the dataset
#print(wiki_data.isnull().sum())

In [ ]:
kaggle

,text,generated
0,Car-free cities have become a subject of incre...,1
1,"Car Free Cities Car-free cities, a concept ga...",1
2,A Sustainable Urban Future Car-free cities ...,1
3,Pioneering Sustainable Urban Living In an e...,1
4,The Path to Sustainable Urban Living In an ...,1
...,...,...
29140,There has been a fuss about the Elector Colleg...,0
29141,Limiting car usage has many advantages. Such a...,0
29142,There's a new trend that has been developing f...,0
29143,As we all know cars are a big part of our soci...,0


In [ ]:
# check seperate the kaggle dataset and perform random sampling to ensure balanced data
kaggle.rename(columns={'generated': 'label'}, inplace=True)
AIG_data = kaggle[kaggle['label'] == 1]
No_AIG_data = kaggle[kaggle['label'] ==0]
print(len(AIG_data), len(No_AIG_data))

11637 17508


In [ ]:
# perform random sampling and ensure balanced data
np.random.seed(42)
# Sample about 10K rows from the wikipedia dataset randomly
AIG_data = AIG_data.sample(frac= 0.25*len(kaggle)/len(AIG_data))
No_AIG_data = No_AIG_data.sample(frac= 0.25*len(kaggle)/len(No_AIG_data))
kaggle_data = pd.concat([AIG_data, No_AIG_data])
# rename the 'generated' column to 'label'
# check if there're missing values in the dataset
print(kaggle_data.isnull().sum())

text     0
label    0
dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
new_essay = pd.read_excel('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_data.xlsx')
label_0_data = new_essay[new_essay['label'] == 0]
label_1_data = new_essay[new_essay['label'] == 1]

# Split each label's data into training and validation sets while maintaining pairing of non-AIG and AIT paraphrased
label_0_train, label_0_val = train_test_split(label_0_data, test_size=0.4, random_state=2024)
label_1_train, label_1_val = train_test_split(label_1_data, test_size=0.4, random_state=2024)

# Combine the training and validation sets for each label
new_essay_train_data = pd.concat([label_0_train, label_1_train])
new_essay_val_data = pd.concat([label_0_val, label_1_val])

## Data Preprocessing

In [ ]:
# Data Preprocessing
!pip install --upgrade pip
!pip install transformers
!pip install ftfy
!pip install ax-platform

In [ ]:
# processing
import numpy as np
import pandas as pd
# text
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from ftfy import fix_text
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def data_preprocessing(df):
    # Remove rows with any missing values
    df = df.dropna()
    # Drop duplicates where both 'text' and 'label' are the same
    df = df.drop_duplicates(subset=['text', 'label'])
    # Drop all entries where 'text' appears more than once (across different labels)
    df = df.drop_duplicates(subset='text', keep=False)
    return df

In [ ]:
#prepare train data
kaggle_data = data_preprocessing(kaggle_data)
wiki_data = data_preprocessing(wiki_data)
# combine the text into corpus
df_list = [kaggle_data, wiki_data]
train_data_v2 = pd.concat(df_list)
kaggle = data_preprocessing(kaggle)

In [ ]:
new_essay_train_data= data_preprocessing(new_essay_train_data)
new_essay_val_data= data_preprocessing(new_essay_val_data)

In [ ]:
kaggle['label'].value_counts()

label
0    16122
1    11218
Name: count, dtype: int64

## Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('averaged_perceptron_tagger')
class TextFeatureExtractor:
    def __init__(self):
        self.stopwords = set(stopwords.words('english'))

    def sentence_length_variation(self, text):
        sentences = sent_tokenize(text)
        sentence_lengths = [len(word_tokenize(sentence)) for sentence in sentences]
        variation = np.std(sentence_lengths)
        return variation

    def count_adjectives(self, text):
        tokens = word_tokenize(text)
        pos_tags = nltk.pos_tag(tokens)
        adjective_count = len([word for word, pos in pos_tags if pos.startswith('JJ')])
        return adjective_count

    def transform(self, df):
        original_columns = df.columns.tolist()
        df['sentence_variation'] = df['text'].apply(self.sentence_length_variation)
        df['word_count'] = df['text'].apply(lambda x: len(str(x).lower().split()))
        df['stop_word_count'] = df['text'].apply(lambda x: len([w for w in str(x).lower().split() if w in self.stopwords]))
        df['%stop_word_total'] = df['stop_word_count'] / df['word_count']
        df['mean_word_length'] = df['text'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
        df['%adjectives_total'] = df['text'].apply(self.count_adjectives) / df['word_count']

        # Columns to keep after transformation
        columns_to_keep = original_columns + ['sentence_variation', '%stop_word_total', 'mean_word_length', '%adjectives_total']
        df = df[columns_to_keep]
        return df

def normalise_text(text):
    text = fix_text(text)
    text = text.lower()  # lowercase
    text = text.translate(str.maketrans('', '', string.punctuation))    # remove punctuation
    text = re.sub(r'\s{2,}', ' ', text)   # replace more than or equal to two white spaces into one white space.
    return text

def remove_stopwords(data):
  stop_words = set(stopwords.words('english'))
  data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
  return data

text_feature_extractor = TextFeatureExtractor()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
kaggle = text_feature_extractor.transform(kaggle)
train_data_v2= text_feature_extractor.transform(train_data_v2)
new_essay_train_data= text_feature_extractor.transform(new_essay_train_data)
new_essay_val_data= text_feature_extractor.transform(new_essay_val_data)

In [ ]:
kaggle['text'] = kaggle['text'].apply(lambda text: fix_text(text))
kaggle['text'] = kaggle['text'].apply(lambda text: normalise_text(text))
train_data_v2['text'] = train_data_v2['text'].apply(lambda text: fix_text(text))
train_data_v2['text'] = train_data_v2['text'].apply(lambda text: normalise_text(text))
new_essay_train_data['text'] = new_essay_train_data['text'].apply(lambda text: fix_text(text))
new_essay_train_data['text'] = new_essay_train_data['text'].apply(lambda text: normalise_text(text))
new_essay_val_data['text'] = new_essay_val_data['text'].apply(lambda text: fix_text(text))
new_essay_val_data['text'] = new_essay_val_data['text'].apply(lambda text: normalise_text(text))

In [ ]:
train_data_v2 = remove_stopwords(train_data_v2)
kaggle= remove_stopwords(kaggle)
new_essay_train_data = remove_stopwords(new_essay_train_data)
new_essay_val_data= remove_stopwords(new_essay_val_data)

In [ ]:
train_data_v2[train_data_v2['%adjectives_total']>0.3]

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
4496,page indexes individual year art pages see als...,0.0,89.810912,0.038298,3.251064,0.442553
6127,tries expr 8 5 4 4 10 3 5 5 8 5 6 6 6 7 4 7 6 ...,0.0,0.000000,0.060606,2.884848,0.309091


In [ ]:
wiki.iloc[6127]

id                                                                33408468
url                      https://en.wikipedia.org/wiki/2011%E2%80%9312%...
title                                                       2011–12 LV Cup
wiki_intro               | tries = {{#expr: \n \n + 8 + 5 + 4 + 4 + 10 ...
generated_intro          | tries = {{#expr: \n \n + tries / 40 }}\n\nTh...
title_len                                                                3
wiki_intro_len                                                         155
generated_intro_len                                                    103
prompt                   200 word wikipedia style introduction on '2011...
generated_text            tries / 40 }}\n\nThe 2011-12 LV Cup was the 8...
prompt_tokens                                                           30
generated_text_tokens                                                  138
Name: 34552, dtype: object

In [ ]:
import nltk
from nltk.corpus import stopwords

def check_stopwords(*dfs):
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    has_stopwords = False
    for df in dfs:
        for text in df['text']:
            words = text.split()
            for word in words:
                if word.lower() in stop_words:
                    has_stopwords = True
                    break
            if has_stopwords:
                break
        if has_stopwords:
            break
    return has_stopwords

has_stopwords = check_stopwords(train_data_v2, kaggle, new_essay_train_data, new_essay_val_data)
if has_stopwords:
    print("At least one of the datasets contains stopwords.")
else:
    print("None of the datasets contain stopwords.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


None of the datasets contain stopwords.


In [ ]:
#prepare test data
essay = pd.read_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/competition_essay.csv')
test_data = essay.drop(columns=['Remark','Wordcount'])
test_data.columns=['text','label']
test_data.head()
def test_transformation (df):
  df= data_preprocessing (df)
  df= text_feature_extractor.transform(df)
  df ['text'] = df['text'].apply(lambda text: fix_text(text))
  df['text'] = df['text'].apply(lambda text: normalise_text(text))
  df=remove_stopwords(df)
  return df
test_data= test_transformation (test_data)

In [ ]:
sorted_new_essay_val_data = new_essay_val_data.sort_index()
sorted_new_essay_val_data

,text,label,sentence_variation,%stop_word_total,mean_word_length,%adjectives_total
6,leaves ivy dance wind outside tapping window s...,0,23.730775,0.439539,4.685221,0.070058
7,courtrooms melancholy ballet leaves ivy dance ...,1,23.674597,0.440628,4.820412,0.065751
8,perfect day green fauxleather chairs stiff blu...,0,18.254857,0.526934,4.237023,0.085211
9,perfect day dreary confines hospital haunting ...,1,7.941484,0.515608,4.682454,0.074273
24,mitigation crime honduras microfinance loans s...,0,10.491831,0.423156,5.239754,0.091189
25,empowering economic development crime mitigati...,1,9.289979,0.339326,6.430337,0.106742
26,marshall plan test progress test progress whet...,0,11.491986,0.384845,5.544367,0.120638
27,marshall plan paradigm american foreign policy...,1,6.744991,0.334934,6.408163,0.152461
28,burmese days democratization us – burma relati...,0,10.211927,0.386200,5.512873,0.113285
29,fostering democratization burma multifaceted a...,1,4.253230,0.363357,6.322416,0.157625


In [ ]:
#train_data_v1.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v1.csv', index=False)
train_data_v2.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/train_preprocessed_v2.csv', index=False)
kaggle.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/kaggle_preprocessed.csv', index=False)

In [ ]:
test_data.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/test_data_preprocessed.csv', index=False)

In [ ]:
new_essay_train_data.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_train_preprocessed.csv', index=False)
new_essay_val_data.to_csv('/content/drive/MyDrive/NUS_MSBA/BT5153_Final Group Project_Shared Folder/Data/new_essay_val_preprocessed.csv', index=False)